In [ ]:
import pandas as pd
import numpy as np
from itertools import product

# Step 1: Load the Data
data = pd.read_csv('2025 DCMP - Match Data.csv')  

# Calculate reef cycles as the sum of t_L2, t_L3, t_L4
data['t_reef'] = data['t_L2'] + data['t_L3'] + data['t_L4']

# Extract key metrics for optimization
teams = data[['team_key', 't_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].copy()
teams['max_cycles'] = teams[['t_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].max(axis=1)

# Points per task (you may adjust these based on actual scoring values)
POINTS = {
    't_L1': 2,       # Points per L1 cycle
    't_reef': 3,     # Points per reef cycle
    't_reefAlgae': 1,  # Points per algae scored
    't_Net': 1,      # Points per algae removed (Net)
    't_Proc': 2      # Points per algae processed
}

# Step 2: Define Function to Calculate Points for a Single Robot
def calculate_points(allocation, robot_data, total_algae_removed):
    """
    Calculate total points based on task allocation and robot capabilities.
    allocation: List of % time spent on [reef, L1, algae].
    robot_data: Row of robot data containing max cycles and points info.
    total_algae_removed: Total algae removed across the alliance.
    """
    reef, L1, algae = allocation
    max_cycles = robot_data['max_cycles']
    
    # Cycles assigned to each task
    reef_cycles = reef / 100 * max_cycles
    L1_cycles = L1 / 100 * max_cycles
    algae_cycles = algae / 100 * max_cycles
    
    # Ensure total cycles don't exceed the maximum possible cycles
    if reef_cycles + L1_cycles + algae_cycles > max_cycles:
        return 0  # Invalid allocation
    
    # Determine the maximum reef cycles based on algae removal
    max_reef = 18 + min(total_algae_removed, 3) * 4 + max(total_algae_removed - 3, 0) * 2
    reef_cycles = min(reef_cycles, max_reef)  # Apply the dynamic reef cycle limit
    
    # Calculate points for each task
    total_points = (
        reef_cycles * POINTS['t_reef'] +
        L1_cycles * POINTS['t_L1'] +
        algae_cycles * POINTS['t_reefAlgae']
    )
    return total_points

# Step 3: Optimize Task Allocation for the Alliance
def optimize_alliance(teams):
    """
    Optimize the task allocation for the alliance of three robots.
    teams: DataFrame containing team data and capabilities.
    """
    best_allocation = None
    best_total_points = 0

    # Generate all possible % allocations (0-100% for each robot and task)
    allocations = product(range(0, 101, 10), repeat=3)
    for combo in product(allocations, repeat=3):  # Three robots, each with an allocation
        if any(sum(alloc) > 100 for alloc in combo):  # Total % cannot exceed 100 per robot
            continue

        # Calculate total algae removed across the alliance
        total_algae_removed = sum([alloc[2] / 100 * teams.iloc[i]['t_reefAlgae'] for i, alloc in enumerate(combo)])
        total_algae_removed = min(total_algae_removed, 9)  # Cap algae removal to 9 cycles

        # Calculate total points for the alliance
        total_points = 0
        for i, allocation in enumerate(combo):
            robot_data = teams.iloc[i]
            total_points += calculate_points(allocation, robot_data, total_algae_removed)

        # Check if this allocation is better
        if total_points > best_total_points:
            best_total_points = total_points
            best_allocation = combo

    return best_allocation, best_total_points

# Step 4: Run the Optimization
# Select the three robots in a given match (e.g., red alliance teams)
red_alliance = ['frc1778', 'frc6831', 'frc2910']
alliance_data = teams[teams['team_key'].isin(red_alliance)]

# Run optimization for the red alliance
best_allocation, best_total_points = optimize_alliance(alliance_data)

# Output the Results
print(f"Best Allocation: {best_allocation}")
print(f"Best Total Points: {best_total_points}")

# Add more details for each robot's task breakdown
for i, robot in enumerate(red_alliance):
    print(f"Robot {robot}: {best_allocation[i]}")


Best Allocation: ((100, 0, 0), (100, 0, 0), (100, 0, 0))
Best Total Points: 81.0
Robot frc1778: (100, 0, 0)
Robot frc6831: (100, 0, 0)
Robot frc2910: (100, 0, 0)


In [ ]:
import pandas as pd
import numpy as np
from itertools import product

# Step 1: Load the Data
data = pd.read_csv('2025 DCMP - Match Data.csv')  

# Calculate reef cycles as the sum of t_L2, t_L3, t_L4
data['t_reef'] = data['t_L2'] + data['t_L3'] + data['t_L4']

# Extract key metrics for optimization
teams = data[['team_key', 't_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].copy()
teams['max_cycles'] = teams[['t_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].max(axis=1)

# Step 2: Define Function to Calculate Total Reef Capacity
def calculate_max_reef(algae_removed):
    """
    Calculate the maximum number of reef cycles based on the number of algae removals.
    algae_removed: Total number of algae removed (t_reefAlgae).
    """
    base_reef = 18
    if algae_removed <= 3:
        return base_reef + 4 * algae_removed
    else:
        return base_reef + 4 * 3 + 2 * (algae_removed - 3)

# Step 3: Define Function to Calculate Total Cycles for an Alliance
def calculate_cycles(allocation, robot_data, total_algae_removed, total_algae_scored, total_reef_cycles):
    """
    Calculate total cycles based on task allocation and robot capabilities.
    allocation: List of % time spent on [reef, L1, algae removal, algae scoring].
    robot_data: Row of robot data containing max cycles and capabilities.
    total_algae_removed: Total algae removed across the alliance.
    total_algae_scored: Total algae scored across the alliance.
    total_reef_cycles: Total reef cycles scored across the alliance.
    """
    reef, L1, algae_remove, algae_score = allocation
    max_cycles = robot_data['max_cycles']
    
    # Cycles assigned to each task
    reef_cycles = reef / 100 * max_cycles
    L1_cycles = L1 / 100 * max_cycles
    algae_remove_cycles = algae_remove / 100 * max_cycles
    algae_score_cycles = algae_score / 100 * max_cycles
    
    # Ensure total cycles don't exceed the maximum possible cycles for this robot
    if reef_cycles + L1_cycles + algae_remove_cycles + algae_score_cycles > max_cycles:
        return 0, 0, 0, 0  # Invalid allocation
    
    # Apply global constraints
    reef_cycles = min(reef_cycles, total_reef_cycles)
    algae_remove_cycles = min(algae_remove_cycles, total_algae_removed)
    algae_score_cycles = min(algae_score_cycles, total_algae_scored)
    
    return reef_cycles, L1_cycles, algae_remove_cycles, algae_score_cycles

# Step 4: Optimize Task Allocation for the Alliance
def optimize_alliance(teams):
    """
    Optimize the task allocation for the alliance of three robots.
    teams: DataFrame containing team data and capabilities.
    """
    best_allocation = None
    best_total_cycles = 0

    # Generate all possible % allocations (0-100% for each robot and task)
    allocations = product(range(0, 101, 10), repeat=4)  # Four tasks: reef, L1, algae remove, algae score
    for combo in product(allocations, repeat=3):  # Three robots, each with an allocation
        if any(sum(alloc) > 100 for alloc in combo):  # Total % cannot exceed 100 per robot
            continue

        # Initialize global constraints
        total_algae_removed = 6  # Max algae removals
        total_algae_scored = 9  # Max algae scored
        total_reef_cycles = 36  # Max reef cycles

        # Calculate total cycles for the alliance
        total_cycles = 0
        for i, allocation in enumerate(combo):
            robot_data = teams.iloc[i]
            reef, L1, algae_remove, algae_score = calculate_cycles(
                allocation, robot_data, total_algae_removed, total_algae_scored, total_reef_cycles
            )
            
            # Update global constraints
            total_algae_removed -= algae_remove
            total_algae_scored -= algae_score
            total_reef_cycles -= reef
            
            # Add up total cycles
            total_cycles += reef + L1 + algae_remove + algae_score

        # Check if this allocation is better
        if total_cycles > best_total_cycles:
            best_total_cycles = total_cycles
            best_allocation = combo

    return best_allocation, best_total_cycles

# Step 5: Run the Optimization
# Select the three robots in a given match (e.g., red alliance teams)
red_alliance = ['frc1778', 'frc6831', 'frc2910']
alliance_data = teams[teams['team_key'].isin(red_alliance)]

# Run optimization for the red alliance
best_allocation, best_total_cycles = optimize_alliance(alliance_data)

# Output the Results
print(f"Best Allocation: {best_allocation}") 
print(f"Best Total Cycles: {best_total_cycles}")

# Add more details for each robot's task breakdown
for i, robot in enumerate(red_alliance):
    print(f"Robot {robot}: {best_allocation[i]}")


In [3]:
import pandas as pd
import numpy as np
from itertools import product
from functools import lru_cache

# Step 1: Load the Data
data = pd.read_csv('2025 DCMP - Match Data.csv')  

# Calculate reef cycles as the sum of t_L2, t_L3, t_L4
data['t_reef'] = data['t_L2'] + data['t_L3'] + data['t_L4']

# Extract key metrics for optimization
teams = data[['team_key', 't_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].copy()
teams['max_cycles'] = teams[['t_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].max(axis=1)

# Precompute possible reef capacity based on algae removal (for faster lookup)
@lru_cache(None)
def precompute_max_reef():
    """
    Precompute max reef cycles based on number of algae removals.
    """
    max_reef_lookup = {}
    for algae_removed in range(0, 7):  # 0 to 6 algae removals
        base_reef = 18
        if algae_removed <= 3:
            max_reef = base_reef + 4 * algae_removed
        else:
            max_reef = base_reef + 4 * 3 + 2 * (algae_removed - 3)
        max_reef_lookup[algae_removed] = max_reef
    return max_reef_lookup

# Precomputed reef capacity lookup table
MAX_REEF_LOOKUP = precompute_max_reef()

# Step 2: Optimize Task Allocation for the Alliance
def optimize_alliance(teams):
    """
    Optimize the task allocation for the alliance of three robots.
    teams: DataFrame containing team data and capabilities.
    """
    best_allocation = None
    best_total_cycles = 0

    # Generate all possible % allocations (0, 20, ..., 100% for each robot and task)
    allocations = product(range(0, 101, 20), repeat=4)  # Four tasks: reef, L1, algae remove, algae score
    for combo in product(allocations, repeat=3):  # Three robots, each with an allocation
        if any(sum(alloc) > 100 for alloc in combo):  # Total % cannot exceed 100 per robot
            continue

        # Initialize global constraints
        total_algae_removed = 6  # Max algae removals
        total_algae_scored = 9  # Max algae scored
        total_reef_cycles = 36  # Max reef cycles

        # Calculate total cycles for the alliance
        total_cycles = 0
        valid = True
        robot_cycles = []

        for i, allocation in enumerate(combo):
            reef, L1, algae_remove, algae_score = allocation
            max_cycles = teams.iloc[i]['max_cycles']
            
            # Cycles assigned to each task
            reef_cycles = reef / 100 * max_cycles
            L1_cycles = L1 / 100 * max_cycles
            algae_remove_cycles = algae_remove / 100 * max_cycles
            algae_score_cycles = algae_score / 100 * max_cycles

            # Check robot-specific constraints
            if reef_cycles + L1_cycles + algae_remove_cycles + algae_score_cycles > max_cycles:
                valid = False
                break

            # Apply global constraints
            reef_cycles = min(reef_cycles, total_reef_cycles)
            algae_remove_cycles = min(algae_remove_cycles, total_algae_removed)
            algae_score_cycles = min(algae_score_cycles, total_algae_scored)

            # Update global constraints
            total_reef_cycles -= reef_cycles
            total_algae_removed -= algae_remove_cycles
            total_algae_scored -= algae_score_cycles

            # Store this robot's cycles
            robot_cycles.append({
                'Reef Cycles': reef_cycles,
                'L1 Cycles': L1_cycles,
                'Algae Remove Cycles': algae_remove_cycles,
                'Algae Score Cycles': algae_score_cycles
            })

            # Add up total cycles
            total_cycles += reef_cycles + L1_cycles + algae_remove_cycles + algae_score_cycles

        # If constraints were violated, skip this combination
        if not valid:
            continue

        # Check if this allocation is better
        if total_cycles > best_total_cycles:
            best_total_cycles = total_cycles
            best_allocation = robot_cycles

    return best_allocation, best_total_cycles

# Step 3: Run the Optimization
# Select the three robots in a given match (e.g., red alliance teams)
red_alliance = ['frc1778', 'frc6831', 'frc2910']
alliance_data = teams[teams['team_key'].isin(red_alliance)]

# Run optimization for the red alliance
best_allocation, best_total_cycles = optimize_alliance(alliance_data)

# Step 4: Display the Results
print(f"Best Total Cycles: {best_total_cycles}\n")
print("Best Allocation (by robot):")

# Create a summary table
allocation_summary = []
for i, robot in enumerate(red_alliance):
    allocation_summary.append({
        'Robot': robot,
        'Reef Cycles': best_allocation[i]['Reef Cycles'],
        'L1 Cycles': best_allocation[i]['L1 Cycles'],
        'Algae Remove Cycles': best_allocation[i]['Algae Remove Cycles'],
        'Algae Score Cycles': best_allocation[i]['Algae Score Cycles']
    })

# Convert summary to a DataFrame for cleaner output
summary_df = pd.DataFrame(allocation_summary)
print(summary_df)

# Save the summary to a CSV file for further analysis
summary_df.to_csv('allocation_summary.csv', index=False)
print("\nAllocation summary saved to 'allocation_summary.csv'")
print(summary_df)


Best Total Cycles: 27.0

Best Allocation (by robot):
     Robot  Reef Cycles  L1 Cycles  Algae Remove Cycles  Algae Score Cycles
0  frc1778          0.0        0.0                  0.0                 8.0
1  frc6831          0.0        0.0                  4.0                 1.0
2  frc2910          0.0       14.0                  0.0                 0.0

Allocation summary saved to 'allocation_summary.csv'
     Robot  Reef Cycles  L1 Cycles  Algae Remove Cycles  Algae Score Cycles
0  frc1778          0.0        0.0                  0.0                 8.0
1  frc6831          0.0        0.0                  4.0                 1.0
2  frc2910          0.0       14.0                  0.0                 0.0


In [1]:
import pandas as pd
import numpy as np
from itertools import product
from functools import lru_cache

# Step 1: Load the Data
data = pd.read_csv('2025 DCMP - Match Data.csv')  

# Calculate reef cycles as the sum of t_L2, t_L3, t_L4
data['t_reef'] = data['t_L2'] + data['t_L3'] + data['t_L4']

# Extract key metrics for optimization
teams = data[['team_key', 't_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].copy()
teams['max_cycles'] = teams[['t_L1', 't_reef', 't_reefAlgae', 't_Net', 't_Proc']].max(axis=1)

# Precompute possible reef and algae scoring capacity based on algae removals
@lru_cache(None)
def precompute_max_capacity():
    """
    Precompute max reef and algae scoring cycles based on algae removals.
    """
    max_capacity_lookup = {}
    for algae_removed in range(0, 7):  # 0 to 6 algae removals
        max_reef = 18 + 3 * algae_removed  # +3 reef cycles per algae removal
        max_algae_score = 3 + algae_removed  # +1 algae score per algae removal
        max_capacity_lookup[algae_removed] = (max_reef, max_algae_score)
    return max_capacity_lookup

# Precomputed capacity lookup table
MAX_CAPACITY_LOOKUP = precompute_max_capacity()

# Step 2: Define the Scoring Function
def calculate_points(allocation, robot_data, total_algae_removed, total_reef_cycles, total_algae_score_cycles):
    """
    Calculate total points for a single robot based on task allocation and global constraints.
    allocation: % time spent on [reef, L1, algae remove, algae score].
    robot_data: Data for a single robot.
    total_algae_removed: Total algae removal capacity remaining.
    total_reef_cycles: Total reef cycles capacity remaining.
    total_algae_score_cycles: Total algae scoring capacity remaining.
    """
    reef, L1, algae_remove, algae_score = allocation
    max_cycles = robot_data['max_cycles']
    
    # Cycles assigned to each task
    reef_cycles = reef / 100 * max_cycles
    L1_cycles = L1 / 100 * max_cycles
    algae_remove_cycles = algae_remove / 100 * max_cycles
    algae_score_cycles = algae_score / 100 * max_cycles

    # Check robot-specific constraints
    if reef_cycles + L1_cycles + algae_remove_cycles + algae_score_cycles > max_cycles:
        return 0, 0, 0, 0, 0  # Invalid allocation

    # Apply global constraints
    reef_cycles = min(reef_cycles, total_reef_cycles)
    algae_remove_cycles = min(algae_remove_cycles, total_algae_removed)
    algae_score_cycles = min(algae_score_cycles, total_algae_score_cycles)

    # Calculate points
    total_points = (
        reef_cycles * 4 +  # Reef = 4 points per cycle
        L1_cycles * 2 +    # L1 = 2 points per cycle
        algae_score_cycles * 4  # Algae scoring = 4 points per cycle
    )

    return total_points, reef_cycles, L1_cycles, algae_remove_cycles, algae_score_cycles

# Step 3: Optimize Task Allocation for the Alliance
def optimize_alliance(teams):
    """
    Optimize the task allocation for the alliance of three robots to maximize points.
    teams: DataFrame containing team data and capabilities.
    """
    best_allocation = None
    best_total_points = 0

    # Generate all possible % allocations (0, 20, ..., 100% for each robot and task)
    allocations = product(range(0, 101, 20), repeat=4)  # Four tasks: reef, L1, algae remove, algae score
    for combo in product(allocations, repeat=3):  # Three robots, each with an allocation
        if any(sum(alloc) > 100 for alloc in combo):  # Total % cannot exceed 100 per robot
            continue

        # Initialize global constraints
        total_algae_removed = 6  # Max algae removals
        total_reef_cycles = 18  # Initial max reef cycles
        total_algae_score_cycles = 3  # Initial max algae score cycles

        # Adjust reef and algae scoring capacity based on algae removal
        total_points = 0
        valid = True
        robot_allocations = []

        for i, allocation in enumerate(combo):
            robot_data = teams.iloc[i]
            points, reef_cycles, L1_cycles, algae_remove_cycles, algae_score_cycles = calculate_points(
                allocation, robot_data, total_algae_removed, total_reef_cycles, total_algae_score_cycles
            )
            
            # If the allocation is invalid, skip this combination
            if points == 0:
                valid = False
                break

            # Update global constraints
            total_reef_cycles -= reef_cycles
            total_algae_removed -= algae_remove_cycles
            total_algae_score_cycles -= algae_score_cycles

            # Adjust reef and algae scoring capacity based on algae removed
            max_reef, max_algae_score = MAX_CAPACITY_LOOKUP[6 - int(total_algae_removed)]
            total_reef_cycles = min(total_reef_cycles, max_reef)
            total_algae_score_cycles = min(total_algae_score_cycles, max_algae_score)

            # Track robot-specific allocations
            robot_allocations.append({
                'Reef Cycles': reef_cycles,
                'L1 Cycles': L1_cycles,
                'Algae Remove Cycles': algae_remove_cycles,
                'Algae Score Cycles': algae_score_cycles,
                'Points': points
            })

            # Add to total points
            total_points += points

        # If constraints were violated, skip this combination
        if not valid:
            continue

        # Check if this allocation is better
        if total_points > best_total_points:
            best_total_points = total_points
            best_allocation = robot_allocations

    return best_allocation, best_total_points

# Step 4: Run the Optimization
# Select the three robots in a given match (e.g., red alliance teams)
red_alliance = ['frc1778', 'frc6831', 'frc2910']
alliance_data = teams[teams['team_key'].isin(red_alliance)]

# Run optimization for the red alliance
best_allocation, best_total_points = optimize_alliance(alliance_data)

# Step 5: Display the Results
print(f"Best Total Points: {best_total_points}\n")
print("Best Allocation (by robot):")

# Create a summary table
allocation_summary = []
for i, robot in enumerate(red_alliance):
    allocation_summary.append({
        'Robot': robot,
        'Reef Cycles': best_allocation[i]['Reef Cycles'],
        'L1 Cycles': best_allocation[i]['L1 Cycles'],
        'Algae Remove Cycles': best_allocation[i]['Algae Remove Cycles'],
        'Algae Score Cycles': best_allocation[i]['Algae Score Cycles'],
        'Points': best_allocation[i]['Points']
    })

# Convert summary to a DataFrame for cleaner output
summary_df = pd.DataFrame(allocation_summary)
print(summary_df)

# Save the summary to a CSV file for further analysis
summary_df.to_csv('allocation_summary_points.csv', index=False)
print("\nAllocation summary saved to 'allocation_summary_points.csv'")


Best Total Points: 95.6

Best Allocation (by robot):
     Robot  Reef Cycles  L1 Cycles  Algae Remove Cycles  Algae Score Cycles  \
0  frc1778          0.0        4.8                  0.0                 3.0   
1  frc6831          4.0        1.0                  0.0                 0.0   
2  frc2910         14.0        0.0                  0.0                 0.0   

   Points  
0    21.6  
1    18.0  
2    56.0  

Allocation summary saved to 'allocation_summary_points.csv'
